In [1]:
from dataclasses import dataclass
from transformers.utils import get_json_schema


def get_weather(city: str, unit: str = "celsius"):
    """
    Get the current weather for a city

    Args:
        city: The name of the city
        unit: Temperature unit (choices: ["celsius", "fahrenheit"])
    """
    return f"Weather in {city}: 22 degrees {unit}"

@dataclass
class ToolCall:
    id:str
    name:str
    arguments:str

@dataclass
class AssistantResponse:
    content:str

import json
from lm import LM

# llm = LM(model="vllm:/home/ng6309/datascience/santhosh/models/Qwen3-14B",api_base="http://localhost:8000")
llm = LM(model="vllm:/home/ng6309/datascience/santhosh/models/Qwen3-14B")

messages = [{"role": "user", "content": "what is weather in london and canada?, do two parallel tool calls to get the weather in both cities /no_think"}]
tools = [get_weather]

async def gen(
        llm,
        history,
        tools=None
):
    tools = [get_json_schema(x) if callable(x) else x for x in tools] if tools else []
    async for x in llm.stream(messages=history,tools=tools):
        delta = x['choices'][0]['delta']

        if 'tool_calls' in delta:
            tool_call = delta['tool_calls'][0]
            if 'id' in tool_call:
                tool_call_id = tool_call['id']
                tool_call_name = tool_call['function']['name']

            if 'arguments' in tool_call['function']:    
                yield ToolCall(id=tool_call_id,name=tool_call_name,arguments=tool_call['function']['arguments'])
        elif 'content' in delta:
            yield AssistantResponse(content=delta['content'])
        else:
            raise ValueError("Unknown delta format")
        
async for x in gen(llm=llm,history=messages,tools=tools):
    print(x,flush=True)



/home/ng6309/datascience/anand/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssistantResponse(content='')
AssistantResponse(content='<think>')
AssistantResponse(content='\n\n')
AssistantResponse(content='</think>')
AssistantResponse(content='\n\n')
ToolCall(id='chatcmpl-tool-56883ce395f54a4781cb0f93267d0826', name='get_weather', arguments='{"city": "')
ToolCall(id='chatcmpl-tool-56883ce395f54a4781cb0f93267d0826', name='get_weather', arguments='London')
ToolCall(id='chatcmpl-tool-56883ce395f54a4781cb0f93267d0826', name='get_weather', arguments='",')
ToolCall(id='chatcmpl-tool-56883ce395f54a4781cb0f93267d0826', name='get_weather', arguments=' "')
ToolCall(id='chatcmpl-tool-56883ce395f54a4781cb0f93267d0826', name='get_weather', arguments='unit')
ToolCall(id='chatcmpl-tool-56883ce395f54a4781cb0f93267d0826', name='get_weather', arguments='":')
ToolCall(id='chatcmpl-tool-56883ce395f54a4781cb0f93267d0826', name='get_weather', arguments=' "')
ToolCall(id='chatcmpl-tool-56883ce395f54a4781cb0f93267d0826', name='get_weather', arguments='c')
ToolCall(id='chatcmpl-tool-568